In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import folium
import json

In [ ]:
lpd = pd.read_csv('laadpaaldata.csv')

In [ ]:
###Inladen API - kijk naar country code en maxresults
response = requests.get('https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&maxresults=1000&compact=true&verbose=false&key=772ba88b-50db-49f1-b9f1-933c6053edd3')
response.status_code

In [ ]:
###Omzetten naar dictionary
json_lp = response.json()

In [ ]:
###Dataframe bevat kolom die een list zijn. 
#Met json_normalize zet je de eerste kolom om naar losse kolommen
opencharge = pd.json_normalize(json_lp)
#Daarna nog handmatig kijken welke kolommen over zijn in dit geval Connections
#Kijken naar eerst laadpaal op de locatie
#Kan je uitpakken middels:
df4 = pd.json_normalize(opencharge.Connections)
df5 = pd.json_normalize(df4[0])
df5.head()
###Bestanden samenvoegen
ocm = pd.concat([opencharge, df5], axis=1)
ocm.head()

In [ ]:
###Inladen API - kijk naar country code en maxresults
response_ev = requests.get('https://opendata.rdw.nl/resource/8wbe-pu7d.json')
response_ev.status_code

In [ ]:
###Omzetten naar dictionary
json_ev = response_ev.json()

In [ ]:
#build dataframe from json_ev
rdw = pd.DataFrame(json_ev)

In [ ]:
ocm.info()

In [ ]:
ocm.head()

In [ ]:
rdw.info()

In [ ]:
rdw

In [ ]:
lpd.info()

In [ ]:
lpd

In [ ]:
ocm.columns = ocm.columns.str.replace('.', '_') 
ocm.columns = ocm.columns.str.replace(' ', '_')

ocm = ocm.loc[:,~ocm.columns.duplicated()].copy()
ocm = ocm[ocm['Quantity'].notna()]
ocm = ocm[ocm['NumberOfPoints'] != 0]
ocm = ocm[ocm['AddressInfo_Latitude'] > 50]
ocm = ocm.drop(columns=['Connections','AddressInfo_DistanceUnit', 'GeneralComments', 'AddressInfo_RelatedURL', 'UsageCost', 'AddressInfo_AddressLine2', 'AddressInfo_AccessComments', 
                        'AddressInfo_ContactTelephone1', 'AddressInfo_ContactEmail', 'AddressInfo_ContactTelephone2', 'OperatorsReference', 'DataProvidersReference',
                        'Reference', 'Amps', 'Voltage', 'Comments'])
ocm.columns = map(str.lower, ocm.columns)
rdw.columns = map(str.lower, rdw.columns)
lpd.columns = map(str.lower, lpd.columns)

ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace('[a-zA-Z]', '')
ocm['addressinfo_postcode'] = ocm['addressinfo_postcode'].str.replace(' ', '')


In [ ]:
ocm['addressinfo_stateorprovince'].isna().sum()

In [ ]:
ev = rdw.loc[(rdw['brandstof'].str.contains('E')) & (rdw['extern_oplaadbaar'] == 'J')]


In [ ]:
merge = pd.merge(ocm, ev, left_on='addressinfo_postcode', right_on='postcode', how='outer')

In [ ]:
m = folium.Map(location=[52.091684, 5.114055], zoom_start=7)


In [ ]:
import geopandas as gpd
import geojson

In [ ]:
# r =  requests.get('https://data.opendatasoft.com/api/v2/catalog/datasets/georef-netherlands-postcode-pc4%40public/exports/geojson?limit=-1&offset=0&lang=nl&timezone=Europe%2FBerlin')

In [31]:
geofile = gpd.read_file('https://data.opendatasoft.com/api/v2/catalog/datasets/georef-netherlands-postcode-pc4%40public/exports/geojson?limit=-1&offset=0&lang=nl&timezone=Europe%2FBerlin')

In [ ]:
# geofile = r.json()

In [38]:
rdw['postcode'] = rdw['postcode'].astype('str')
rdw['aantal'] = rdw['aantal'].astype('str')
geofile['pc4_code'] = geofile['pc4_code'].astype('str')

In [41]:
folium.Choropleth(
    geo_data=geofile.to_json(),
    name="geometry",
    data=rdw,
    columns=["postcode", 'aantal'],
    key_on="feature.properties.gem_name",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
).add_to(m)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
m